# Implementation Copyright
## https://github.com/owruby

In [1]:
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from tqdm import tqdm
from datasets import load_dataset
from models import ShakePyramidNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from sklearn.model_selection import train_test_split

In [2]:
# GPU Device
gpu_id = '0,1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %s:" %(gpu_id), use_cuda)

GPU device 0,1,2: True


In [3]:
pretrained = ''
resume = ''

In [4]:
# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# train params
epochs = 1800
batch_size = 1800
init_lr = 0.5
weight_decay = 1e-5
labels = 10
momentum = 0.9

# ShakeDrop params
depth = 110
alpha = 270

In [5]:
train_loader, test_loader = load_dataset(10, batch_size)

Files already downloaded and verified


In [6]:
checkpoint = './log/' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 8

In [7]:
model = ShakePyramidNet(depth=depth, alpha=alpha, label=labels)

In [8]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))
model = nn.DataParallel(model)

    Total params: 28.49M


In [9]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

In [10]:
optimizer = optim.SGD(model.parameters(), lr=init_lr, momentum=momentum, weight_decay=weight_decay, nesterov=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
step_lr = torch.optim.lr_scheduler.StepLR(optimizer, 400, gamma=0.1, last_epoch=-1)

In [11]:
loss_func = nn.CrossEntropyLoss().cuda()

In [ ]:
for e in range(epochs):
    model.train()
    train_loss, train_top1 = AverageMeter(), AverageMeter()
    bar = tqdm(total=len(train_loader), leave=False)
    for x, t in train_loader:
        x, t = Variable(x.cuda()), Variable(t.cuda())
        y = model(x)
        loss = loss_func(y, t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        err1 = accuracy(y.data, t, topk=(1,))
        train_loss.update(float(loss.data), x.size(0))
        train_top1.update(float(err1[0]), x.size(0))
        bar.set_description("Epoch: {:d} / {:d}, LR:{:.6f}, Loss: {:.6f}, Top1E: {:.2f}".format(e, epochs, 
            optimizer.param_groups[0]["lr"], train_loss.avg, train_top1.avg), refresh=True)
        bar.update()
    bar.close()
    scheduler.step()
    step_lr.step()
    model.eval()jj
    test_loss, test_top1 = AverageMeter(), AverageMeter()
    for x, t in tqdm(test_loader, total=len(test_loader), leave=False):
        with torch.no_grad():
            x, t = Variable(x.cuda()), Variable(t.cuda())
            y = model(x)
            loss = loss_func(y, t)
            err1 = accuracy(y.data, t, topk=(1,))
            test_loss.update(float(loss.data), x.size(0))
            test_top1.update(float(err1[0]), x.size(0))

    if (e + 1) % 100 == 0:
        torch.save({
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict()
        }, os.path.join(checkpoint, "{}.tar".format(e + 1)))

    lr = optimizer.param_groups[0]["lr"]
    logger.append([lr, train_loss.avg, test_loss.avg, train_top1.avg, test_top1.avg])

100%|██████████| 6/6 [00:07<00:00,  1.27s/it]                                                               